In [1]:
import os
os.chdir('../')

In [2]:
pwd

'c:\\Users\\asbpi\\Desktop\\ASB\\Data_Science\\Projects\\MY Projects\\Stroke-Risk-Prediction'

In [3]:
# Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    data_dir: Path
    status_file: str
    all_schema: dict

In [5]:
# Configuration Manager

In [6]:
from stroke_risk.constants import *
from stroke_risk.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            status_file=config.status_file,
            data_dir = config.data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [8]:
# Components

In [9]:
import os
import pandas as pd
from stroke_risk import logger

In [18]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"COLUMNS: {validation_status}\n")
                else:
                    validation_status = True
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"COLUMNS: {validation_status}\n")
        except Exception as e:
            raise e
    

    def validate_all_datatypes(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.data_dir)
            all_cols = list(data.columns)
            all_schema = self.config.all_schema

            for col in all_cols:
                if all_schema[col] != data[col].dtype.name:
                    validation_status = False
                    with open(self.config.status_file, 'a') as f:
                        f.write(f"{col}: {validation_status}\n")
                    
                    logger.info(f'{col} = {all_schema[col]}: {data[col].dtype.name}')
                else:
                    validation_status = True
                    with open(self.config.status_file, 'a') as f:
                        f.write(f"{col}: {validation_status}\n")
        except Exception as e:
            raise e
    

    def check_validation(self)-> bool:
        try:
            with open(self.config.status_file, 'r') as f:
                status = [stat.strip() for stat in f.readlines()]
                f.close()

            status = {key.strip(): value.strip().lower()=='true' for key,value in [stat.split(':') for stat in status]}

            if all(status.values()):
                logger.info('All Validation Passed')
            else:
                logger.info('Validation Failed')
        except Exception as e:
            raise e

In [11]:
# Pipeline

In [23]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
    data_validation.validate_all_datatypes()
    data_validation.check_validation()
except Exception as e:
    raise e

[2023-12-25 20:27:04,370: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-25 20:27:04,374: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-25 20:27:04,378: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-25 20:27:04,379: INFO: common: created directory at: artifacts]
[2023-12-25 20:27:04,381: INFO: common: created directory at: artifacts/data_validation]
[2023-12-25 20:27:04,437: INFO: 2033076008: All Validation Passed]
